<a href="https://colab.research.google.com/github/ruslanmv/Automatic-Fine-Tunning/blob/master/FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [ruslanmv/Automatic-Fine-Tunning](https://github.com/ruslanmv/Automatic-Fine-Tunning/blob/master/FineTune.ipynb)

In [1]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [2]:
#@markdown # Install/Update ruslanmv repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six


blsaphemy = base64.b64decode("ZWJ1aQ==").decode('ascii')

if not os.path.exists("/content/gdrive"):
    print('\033[1;31mGdrive not connected, using temporary colab storage ...')
    time.sleep(4)
    mainpth = "MyDrive"
    !mkdir -p /content/gdrive/$mainpth
    Shared_Drive = ""

if Shared_Drive != "" and not os.path.exists("/content/gdrive/Shareddrives"):
    print('\033[1;31mShared drive not detected, using default MyDrive')
    mainpth = "MyDrive"

with capture.capture_output() as cap:
    def inf(msg, style, wdth):
        inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
        display(inf)
    fgitclone = "git clone --depth 1"
    !mkdir -p /content/gdrive/$mainpth/llm
    # Define the path
    main_path =f"/content/gdrive/{mainpth}/"
    !git clone -q --branch master https://github.com/ruslanmv/Automatic-Fine-Tunning /content/gdrive/$mainpth/llm/Automatic-Fine-Tunning
    os.environ['TRANSFORMERS_CACHE'] = f"/content/gdrive/{mainpth}/llm/Automatic-Fine-Tunning/{blsaphemy}/cache"
    os.environ['TORCH_HOME'] = f"/content/gdrive/{mainpth}/llm/Automatic-Fine-Tunning/{blsaphemy}/cache"
    cache_dir = os.environ['TRANSFORMERS_CACHE']
    !mkdir -p /content/gdrive/{mainpth}/llm/Automatic-Fine-Tunning/{blsaphemy}/repositories
    !git clone https://github.com/ruslanmv/Automatic-Fine-Tunning /content/gdrive/{mainpth}/llm/Automatic-Fine-Tunning/{blsaphemy}/repositories/Automatic-Fine-Tunningebui-assets

with capture.capture_output() as cap:
    %cd /content/gdrive/{mainpth}/llm/Automatic-Fine-Tunning/{blsaphemy}/repositories/Automatic-Fine-Tunningebui-assets

    !git reset --hard
    !git checkout master
    time.sleep(1)
    !git pull

clear_output()
inf('\u2714 Done', 'success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [3]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/
  !wget -q -i https://github.com/ruslanmv/Automatic-Fine-Tunning/raw/master/Dependencies/requirements.txt
  !pip install -r requirements.txt
clear_output()
def reload_environment():
    # Kernel restart logic (may not work consistently within Jupyter Notebook)
    try:
      from IPython import get_ipython
      get_ipython().kernel.do_shutdown(restart=True)
      print("Kernel restarted. Packages should be reloaded.")
    except Exception as e:
      print(f"Kernel restart failed: {e}")
      print("Consider manually restarting the kernel or your Jupyter Notebook server.")
#reload_environment()

inf('\u2714 Done','success', '50px')
#@markdown ---


Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

## Load Packages

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re
import os

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Definitions

In [5]:
# base model from huggingFace or path to model
base_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# New model name
new_model = "Medical-Mixtral-7B-v1.5k"

## Loading Data

In [6]:
# Access the environment variable
#secret_hf = os.environ.get('HF_TOKEN')
from google.colab import userdata
secret_hf = userdata.get('HF_TOKEN')
# Then you can use the token in your command
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
dataset_ = load_dataset("ruslanmv/ai-medical-chatbot")
train_data = dataset_["train"]
# For this demo, let's choose the first 1500 dialogues
num=1500
#num=256500
#df = pd.DataFrame(train_data[:num])
df = pd.DataFrame(train_data[::])
df = df[["Description", "Doctor"]].rename(columns={"Description": "question", "Doctor": "answer"})
# Clean the question and answer columns
df['question'] = df['question'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['answer'] = df['answer'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
# Assuming your DataFrame is named 'df' and the column is named 'df' and the column is named 'question'
df['question'] = df['question'].str.lstrip('Q. ')

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [ ]:
df.head()

,question,answer
0,What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...
1,What should I do to reduce my weight gained du...,Hi. You have really done well with the hypothy...
2,"I have started to get lots of acne on my face,...",Hi there Acne has multifactorial etiology. Onl...
3,Why do I have uncomfortable feeling between th...,Hello. The popping and discomfort what you fel...
4,My symptoms after intercourse threatns me even...,Hello. The HIV test uses a finger prick blood ...


In [ ]:
df_train = df.iloc[:num, :]
df_test = df.iloc[num:num+100, :]

In [ ]:
# Save the train dataframe to a CSV file
df_train.to_csv('train.csv', index=False)

# Save the test dataframe to a CSV file
df_test.to_csv('test.csv', index=False)


df=df_train

## Formatting Your Fine-tuning Data
There are various ways to format your data for fine-tuning

Prompts provide context and guide the LLM towards the desired task. The code showcases creating prompts for question-answering tasks with placeholders for questions and answers.

In [ ]:
# build training dataset with the right format
df['text'] = '<s>[INST]@Enlighten. ' + df['question'] +'[/INST]'+ df['answer'] + '</s>'

In [ ]:
# remove columns
df=df.drop(['question','answer'],axis=1)

In [ ]:
df.head()

,text
0,<s>[INST]@Enlighten. What does abutment of the...
1,<s>[INST]@Enlighten. What should I do to reduc...
2,<s>[INST]@Enlighten. I have started to get lot...
3,<s>[INST]@Enlighten. Why do I have uncomfortab...
4,<s>[INST]@Enlighten. My symptoms after interco...


In [ ]:
# convert to dataset object
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
dataset = Dataset(pa.Table.from_pandas(df))
dataset

Dataset({
    features: ['text'],
    num_rows: 1500
})

## Downloading and Initializing Mixtral 8x7b

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

('<s>', '</s>')

In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, cache_dir=cache_dir)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [ ]:
# count trainging tokens
from transformers import LlamaTokenizer
tokenizer_ = LlamaTokenizer.from_pretrained("cognitivecomputations/dolphin-llama2-7b",
                                            cache_dir=cache_dir)
tokens = tokenizer_.tokenize(dataset.to_pandas().to_string())
len(tokens)

497617

# Fine-Tune

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.138600
2,2.117100
3,1.830600
4,1.926200
5,2.169500
6,2.018900
7,2.036500
8,1.970300
9,2.003400
10,1.735400


Checkpoint destination directory ./results/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To 

TrainOutput(global_step=375, training_loss=1.3934035609165827, metrics={'train_runtime': 1962.165, 'train_samples_per_second': 0.764, 'train_steps_per_second': 0.191, 'total_flos': 6.459152599754342e+16, 'train_loss': 1.3934035609165827, 'epoch': 1.0})

## Save and push the adapter to HF


In [12]:
# New model name
new_model = "Medical-Mixtral-7B-v1.5k"
# Save the fine-tuned model
save_path = os.path.join(main_path, "llm", "Automatic-Fine-Tuning", "models", new_model)

In [ ]:

trainer.model.save_pretrained(save_path)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixtralForCausalLM(
      (model): MixtralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MixtralDecoderLayer(
            (self_attn): MixtralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [ ]:
trainer.model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ruslanmv/Medical-Mixtral-7B-v1.5k/commit/7a4d908b382ce3931c32de5798ac6ebe1332127e', commit_message='Upload model', commit_description='', oid='7a4d908b382ce3931c32de5798ac6ebe1332127e', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
tokenizer.save_pretrained(save_path)

('/content/gdrive/MyDrive/llm/Automatic-Fine-Tuning/models/Medical-Mixtral-7B-v1.5k/tokenizer_config.json',
 '/content/gdrive/MyDrive/llm/Automatic-Fine-Tuning/models/Medical-Mixtral-7B-v1.5k/special_tokens_map.json',
 '/content/gdrive/MyDrive/llm/Automatic-Fine-Tuning/models/Medical-Mixtral-7B-v1.5k/tokenizer.model',
 '/content/gdrive/MyDrive/llm/Automatic-Fine-Tuning/models/Medical-Mixtral-7B-v1.5k/added_tokens.json',
 '/content/gdrive/MyDrive/llm/Automatic-Fine-Tuning/models/Medical-Mixtral-7B-v1.5k/tokenizer.json')

In [13]:
tokenizer.push_to_hub(new_model)

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ruslanmv/Medical-Mixtral-7B-v1.5k/commit/084a48d151aca5fc78084eec89f06d55d89136d8', commit_message='Upload tokenizer', commit_description='', oid='084a48d151aca5fc78084eec89f06d55d89136d8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import json
# Create a Model Card
model_card = {
  "Model Name": "Medical-Mixtral-7B-v1.5k",
  "Description": "Fine-tuned Mixtral model for answering medical assistance questions. This model is a novel version of mistralai/Mixtral-8x7B-Instruct-v0.1, adapted to a subset of 1.5k records from the AI Medical Chatbot dataset, which contains 250k records (https://huggingface.co/datasets/ruslanmv/ai-medical-chatbot). The purpose of this model is to provide a ready chatbot to answer questions related to medical assistance.",
  "Intended Use": "This model is intended for providing assistance and answering questions related to medical inquiries. It is suitable for use in chatbot applications where users seek medical advice, information, or assistance.",
  "Example Usage": "```python\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\n\n# Define the name of your fine-tuned model\nfinetuned_model = 'ruslanmv/Medical-Assistance-Mixtral-7B-v1.5k'\n\n# Load tokenizer\ntokenizer = AutoTokenizer.from_pretrained(finetuned_model, trust_remote_code=True)\n\n# Load the model with the provided adapter configuration and weights\nmodel_pretrained = AutoModelForCausalLM.from_pretrained(finetuned_model, trust_remote_code=True, torch_dtype=torch.float16, cache_dir=cache_dir)\n\nmessages = [\n    {'role': 'user', 'content': 'What should I do to reduce my weight gained due to genetic hypothyroidism?'},\n    {'role': 'assistant', 'content': ''},\n]\n\ninput_ids = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')\n\noutputs = model_pretrained.generate(input_ids, max_new_tokens=500)\nprint(tokenizer.decode(outputs[0], skip_special_tokens=True))\n```",
  "Training Data": {
    "Dataset Name": "AI Medical Chatbot",
    "Dataset URL": "https://huggingface.co/datasets/ruslanmv/ai-medical-chatbot",
    "Dataset Size": "250k records",
    "Subset Used": "1.5k records"
  },
  "Limitations": [
    "The model's performance may vary depending on the complexity and specificity of the medical questions.",
    "The model may not provide accurate answers for every medical query, and users should consult medical professionals for critical healthcare concerns."
  ],
  "Ethical Considerations": [
    "Users should be informed that the model's responses are generated based on patterns in the training data and may not always be accurate or suitable for medical decision-making.",
    "The model should not be used as a replacement for professional medical advice or diagnosis.",
    "Sensitive patient data should not be shared with the model, and user privacy should be protected."
  ]
}

# Save the Model Card
model_card_path = os.path.join(save_path, "model_card.json")
with open(model_card_path, "w") as f:
    json.dump(model_card, f)

model.push_to_hub(new_model, model_card=model_card_path)

## Test the model


In [ ]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST]@Enlighten. {question} [/INST]"
  return prompt

In [ ]:
question = "What does abutment of the nerve root mean?"
prompt = build_prompt(question)
result = pipe(prompt)

print(result[0]['generated_text'])

<s>[INST]@Enlighten. What does abutment of the nerve root mean? [/INST]Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor/neurology/neurologist --> Dr. Naveen Kumar N. --> For more information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor/neurology/neurologist --> Dr. Naveen Kumar N. -->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->--->


In [ ]:
#test_path="test.csv"
test_path="train.csv"

In [ ]:
df_test=pd.read_csv(test_path)

In [ ]:
df_test.head()

,question,answer
0,What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...
1,What should I do to reduce my weight gained du...,Hi. You have really done well with the hypothy...
2,"I have started to get lots of acne on my face,...",Hi there Acne has multifactorial etiology. Onl...
3,Why do I have uncomfortable feeling between th...,Hello. The popping and discomfort what you fel...
4,My symptoms after intercourse threatns me even...,Hello. The HIV test uses a finger prick blood ...


In [ ]:
#We test only 10 entries
df_test=df_test.head(10)

In [ ]:
df_test

,question,answer
0,What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...
1,What should I do to reduce my weight gained du...,Hi. You have really done well with the hypothy...
2,"I have started to get lots of acne on my face,...",Hi there Acne has multifactorial etiology. Onl...
3,Why do I have uncomfortable feeling between th...,Hello. The popping and discomfort what you fel...
4,My symptoms after intercourse threatns me even...,Hello. The HIV test uses a finger prick blood ...
5,I had a surgery which ended up with some failu...,Hello. If you are saying it is already six mon...
6,Kindly explain about Beta Strep.,Hello. There are lots of bacteria and other or...
7,Kindly suggest a homeopathic medicine to stop ...,Hello. I checked the attached photo (attachmen...
8,What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...
9,Will Nano-Leo give permanent solution for erec...,Hi. For further doubts consult a sexologist on...


In [ ]:
questionCounter = 0
correct = 0
promptEnding = "[/INST]"

# Guide for answering questions
testGuide = 'Answer the following question, at the end of your response say thank you for your query.\n'

# Loop through each row in the DataFrame
for index, row in df_test.iterrows():
    print("#############################")
    questionCounter += 1

    # Build the question prompt
    question = testGuide + row['question'] + "\n"
    print(question)

    # Get the true answer
    truth = row['answer']

    # Build the prompt
    prompt = build_prompt(question)

    # Generate answer
    result = pipe(prompt)
    llmAnswer = result[0]['generated_text']

    # Remove the prompt from the generated answer
    index = llmAnswer.find(promptEnding)
    llmAnswer = llmAnswer[len(promptEnding) + index:]

    print("LLM Answer:")
    print(llmAnswer)

    # Remove spaces from the generated answer
    llmAnswer = llmAnswer.replace(' ', '')

    # Find the option in response
    index = llmAnswer.find('answer:')

    # Find and match the option
    next_char = llmAnswer[index + len('answer:'):][0]
    if next_char in truth:
        correct += 1
        print('correct')
    else:
        print('wrong')

    # Update accuracy
    accuracy = correct / questionCounter
    print(f"Progress: {questionCounter / len(df_test)}")
    print(f"Accuracy: {accuracy}")

#############################
Answer the following question, at the end of your response say thank you for your query.
What does abutment of the nerve root mean?

LLM Answer:
Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor/neurology/neurologist --> M.D. (Neurology), DM (Neurology). For further information consult a neurologist online --> https://www.icliniq.com/ask-a-doctor/neurology/neurologist --> M.D. (Neurology), DM (Neurology). I hope I have clarified your query, do you have any more questions or doubts? I will be happy
correct
Progress: 0.1
Accuracy: 1.0
#############################
Answer the following question, at the end of your response say thank you for your query.
What should I do to reduce my weight gained due to genetic hypothyroidism?

LLM Answer:
Hello. I understand your concern. You have a family history of hypothyroidi

## Medical-Mixtral-7B-v1.5k
Testing model created

In [11]:
#reload_environment()

Kernel restarted. Packages should be reloaded.


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Define the name of your fine-tuned model
finetuned_model = "ruslanmv/Medical-Mixtral-7B-v1.5k"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(finetuned_model,
                                          trust_remote_code=True,
                                          cache_dir=cache_dir)

In [28]:
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [8]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model_pretrained = AutoModelForCausalLM.from_pretrained(
        finetuned_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [9]:
model_pretrained.eval()
## Text Generation Base model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=Fa

In [32]:
def build_prompt(question):
  prompt=f"<s>[INST]@Enlighten. {question} [/INST]"
  return prompt

In [39]:
pipe = pipeline(task="text-generation", model=model_pretrained, tokenizer=tokenizer, max_length=100)

In [40]:
def ask(question):
  promptEnding = "[/INST]"
  # Guide for answering questions
  testGuide = 'Answer the following question, at the end of your response say thank you for your query.\n'
  # Build the question prompt
  question = testGuide + question + "\n"
  print(question)
  # Build the prompt
  prompt = build_prompt(question)
  # Generate answer
  result = pipe(prompt)
  llmAnswer = result[0]['generated_text']
  # Remove the prompt from the generated answer
  index = llmAnswer.find(promptEnding)
  llmAnswer = llmAnswer[len(promptEnding) + index:]
  print("LLM Answer:")
  print(llmAnswer)

In [41]:
question = "My symptoms after intercourse threatns me even after having negative HIV result. Kindly help."

In [42]:
ask(question)

Answer the following question, at the end of your response say thank you for your query.
My symptoms after intercourse threatns me even after having negative HIV result. Kindly help.

LLM Answer:
Hello. I understand your concern. You have been tested for HIV and the result is negative. You have no symptoms of HIV. You have no symptoms of any other sexually transmitted diseases. You have no symptoms of any other diseases.
